<a href="https://colab.research.google.com/github/LarryDCJ/AC_Classifier/blob/master/Classifier_Conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/AC_Classifier

In [ ]:
!git merge --abort

In [45]:
!git reset --merge

In [ ]:
!git add

In [ ]:
! git commit -a

In [34]:
! git config --global user.email "larrydcj@icloud.com"
! git config --global user.name "Larry Cawley"


In [ ]:
! git pull https://github.com/LarryDCJ/AC_Classifier

In [ ]:
! git commit -m "colab updates"

In [ ]:
! git push origin master

In [ ]:
%%capture
!pip install fastbook

In [ ]:
import re
from fastai.data.all import *
from fastai.vision.all import *
from fastai.metrics import error_rate
import pathlib
from IPython.display import Image
from google.colab import drive
import os
from fastai.vision.widgets import *
from utils import *
drive.mount('/content/drive')
config_file = Path("~/.fastai/config.yml").expanduser()

if config_file.exists(): os.remove(config_file)
assert not config_file.exists()
config = Config()
assert config_file.exists()
print(config.d)
config['archive_path'] = Path('drive/archive')
config['data_path'] = Path('drive/data')
config['model_path'] = Path('drive/models')
config['storage_path'] = Path('root/tmp')
config.save()
print(config.d)
print("Please continue")

In [ ]:
path = Path('/content/drive/MyDrive/AC_Classifier/')
print(type(path))
path.ls()
files = get_image_files(path/"images")
def label_func(f): return f[0].isupper()
print(len(files))
print(type(files))

In [ ]:
print(files[0].name)
#def label_func(f): return re.sub("_[^_]+$", "",f) #rename folder based on filename
pat = r'^(.*)_\d+.jpg' #Image Categories from filename

In [ ]:
dls = ImageDataLoaders.from_name_re(path,
                                    files,
                                    pat,
                                    item_tfms=RandomResizedCrop(224, min_scale=0.5),
                                    batch_tfms=aug_transforms())

In [ ]:
dls.train.show_batch(max_n=8,nrows=2, unique=True)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(2, 3e-3)

epoch,train_loss,valid_loss,error_rate,time
0,1.294198,1.009054,0.301759,01:04


epoch,train_loss,valid_loss,error_rate,time
0,0.509148,1.119417,0.313261,01:07
1,0.325311,0.868610,0.263870,01:06
